In [1]:
input_RDD = sc.textFile("/data/students/bigdata-01QYD/Lab6_DBD/Reviews.csv")

In [2]:
def get_userID_userProcucts(line):
    fields = line.split(",")
    #userID = fields[2]
    #uderProduct = fields[1]
    
    return (fields[2], fields[1])
    

In [3]:
input_without_header_RDD = input_RDD.filter(lambda line: "Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text" not in line)

In [4]:
pairs_RDD = input_without_header_RDD.map(get_userID_userProcucts)

In [5]:
#pairs_RDD.top(5)

In [6]:
grouped_iterable_RDD = pairs_RDD.distinct().groupByKey()

In [7]:
#I have to do this to visualize my data because transformation are lazy 
# Materialize the grouped data
grouped_RDD = grouped_iterable_RDD.mapValues(list)

# Collect and print the grouped data
print(grouped_RDD.take(10))

[Stage 1:>                                                          (0 + 3) / 3]

[('A395BORC6FGVXV', ['B000UA0QIQ', 'B002Y7526Y', 'B000U9WZ54']), ('AZOF9E17RGZH8', ['B001GVISJM', 'B001I8EW5C']), ('A2EB6OGOWCRU5H', ['B001EO5QW8']), ('A2GHZ2UTV2B0CD', ['B001EO5QW8', 'B0000TA3QM']), ('A3TWF9DQ6MU87E', ['B005DUM9UQ']), ('AZLONLC8OZPEC', ['B0066DMI6Y']), ('A2VOZX7YBT0D6D', ['B003ZFRKGO', 'B0000V1BKM', 'B000ILA4KW']), ('A3AF72GP4GVRY1', ['B0019CW0HE', 'B001EX42JM']), ('A1DJAAKLPCJRZD', ['B0019CW0HE']), ('A3D1TXE98KRKYO', ['B004K2IHUO', 'B006L4FNB0', 'B00115FPV2', 'B009AVD282', 'B00815FR68', 'B006L4FMR0', 'B002Y1CVZU', 'B000PKMN14'])]


In [8]:
grouped_RDD.values()

PythonRDD[11] at RDD at PythonRDD.scala:53

In [9]:
def pairs_of_products_reviewed_together(line):
    products = line[1]
    Couples = []
    for i in range(len(products)):
        for j in range(i+1,len(products)):
            if products[i]<products[j]:
                Couples.append(((products[i], products[j]), 1))
            else:
                 Couples.append(((products[j], products[i]), 1))
    return Couples


            

In [10]:
products_RDD = grouped_RDD.flatMap(pairs_of_products_reviewed_together)

In [11]:
products_RDD.take(10)

[(('B000UA0QIQ', 'B002Y7526Y'), 1),
 (('B000U9WZ54', 'B000UA0QIQ'), 1),
 (('B000U9WZ54', 'B002Y7526Y'), 1),
 (('B001GVISJM', 'B001I8EW5C'), 1),
 (('B0000TA3QM', 'B001EO5QW8'), 1),
 (('B0000V1BKM', 'B003ZFRKGO'), 1),
 (('B000ILA4KW', 'B003ZFRKGO'), 1),
 (('B0000V1BKM', 'B000ILA4KW'), 1),
 (('B0019CW0HE', 'B001EX42JM'), 1),
 (('B004K2IHUO', 'B006L4FNB0'), 1)]

In [12]:
products_pairs_RDD = products_RDD.reduceByKey(lambda v1, v2: v1 + v2)

In [13]:
products_pairs_RDD.take(10)

[(('B0000V1BKM', 'B003ZFRKGO'), 1),
 (('B0019CW0HE', 'B001EX42JM'), 1),
 (('B00115FPV2', 'B004K2IHUO'), 1),
 (('B004K2IHUO', 'B00815FR68'), 7),
 (('B00115FPV2', 'B006L4FNB0'), 1),
 (('B006L4FNB0', 'B00815FR68'), 7),
 (('B00115FPV2', 'B006L4FMR0'), 1),
 (('B006L4FMR0', 'B009AVD282'), 7),
 (('B002Y1CVZU', 'B00815FR68'), 7),
 (('B002Y1CVZU', 'B006L4FMR0'), 7)]

In [14]:
filtered_products_RDD = products_pairs_RDD.filter(lambda l: l[1]>1)

In [15]:
sorted_RDD = filtered_products_RDD.sortBy(lambda l: l[1], False)

In [16]:
sorted_RDD.take(10)

[(('B002QWHJOU', 'B002QWP89S'), 608),
 (('B0026RQTGE', 'B002QWP8H0'), 608),
 (('B002QWHJOU', 'B002QWP8H0'), 608),
 (('B0026RQTGE', 'B002QWP89S'), 608),
 (('B002QWP89S', 'B002QWP8H0'), 608),
 (('B0026RQTGE', 'B002QWHJOU'), 608),
 (('B000KV61FC', 'B000KV7ZGQ'), 555),
 (('B0026KNQSA', 'B006HYLW32'), 551),
 (('B001RVFEP2', 'B0026KNQSA'), 551),
 (('B001RVFERK', 'B0026KPDG8'), 551)]

In [17]:
#sorted_RDD.saveAsTextFile("Spark_Output/Output_Lab6_1")

In [19]:
maxfreq = sorted_RDD.first()

In [21]:
maxfreq

(('B002QWHJOU', 'B002QWP89S'), 608)